In [2]:
import nltk
import numpy as np
import pandas as pd

import collections
import itertools

In [3]:
%run common.ipynb

1. [ ] checkout [OKSO](https://classifikators.ru/okso);
1. [ ] compare plain distances from "mathematics" to "IT", "javascript", etc and to, say, "machinist";
1. [ ] try manually specifying some skills as dependencies for some jobs and training a net which takes two word embeddings and answers whether one is a dependency for the other;
1. [ ] this isn't going to work out of the box; research what's needed to tune it
1. [ ] datasets available that can be used for adjusting the model?
    - https://www.kaggle.com/niyamatalmass/google-job-skills
    - https://www.kaggle.com/madhab/jobposts
    - https://www.reddit.com/r/dataisbeautiful/comments/25qjpz/how_many_employees_are_moving_between_companies_oc/chjvd0g/
    - https://github.com/hhru/api

In [4]:
from nltk.corpus import wordnet as wn

In [5]:
import pymongo
client = pymongo.MongoClient()
db = client['uni']
vpo1 = db.vpo1

# The Graph: eduprograms versus skills/competences

- [x] Compose the dictionary of educational programs
- [ ] Clean up `hh.csv`
    - [ ] A single record should consist of "description" and a list of "tags" (extracted from `profarea`, etc)
    - [ ] Canonicalize "description"?
- [ ] Prepare subset of the graph, linking a `hh` record to one or more educational programs
- [ ] Pipeline
    - [ ] Prepare autoencoding/wordnet/etc part of the model, to be trained without supervision on "descriptions"
    - [ ] Graph isometric embedding part: implement the loss for violating distances between nodes of prepared graph subset
    - [ ] How to train? Staged training, combined loss, etc?

## Programs

In [11]:
def get_programs():
    df = pd.read_csv(GRADUATES_FILE)
    df = df[['program_code', 'program']]
    df = df.groupby('program_code').first()
    programs = pd.DataFrame({'name': df.program}).sort_index()
    programname_to_id = dict(zip(programs.name, programs.index))
    programcode_to_id = zip(df.index, df.program)
    programcode_to_id = ((code, programname_to_id[name]) for (code, name) in programcode_to_id)
    programcode_to_id = dict(programcode_to_id)
    return programs, programname_to_id, programcode_to_id

programs, name_to_id, code_to_id = get_programs()
programs.to_csv(PROGRAMS_FILE)
pd.DataFrame(code_to_id.items(),
             columns=['code', 'id']).sort_values('code').to_csv(PROGRAMCODE_TO_ID_FILE,
                                                                index=False)
programs

,name
program_code,
01.03.01,Математика
01.03.02,Прикладная математика и информатика
01.03.03,Механика и математическое моделирование
01.03.04,Прикладная математика
01.04.01,Математика
01.04.02,Прикладная математика и информатика
01.04.03,Механика и математическое моделирование
01.04.04,Прикладная математика
01.05.01,Фундаментальные математика и механика


In [12]:
programs.loc[code_to_id['01.03.02']]

name    Прикладная математика и информатика
Name: 01.04.02, dtype: object

In [13]:
programs.loc[code_to_id['58.04.01']]

name    Востоковедение и африканистика
Name: 58.04.01, dtype: object